# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [94]:
import pandas as pd
import numpy as np

In [95]:
cc_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [96]:
cc_scoring.shape

(21525, 12)

In [97]:
# sample of top 5 records
cc_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [98]:
# info about the dataframe
cc_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [99]:
# missing records
cc_scoring.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [100]:
# duplicates
cc_scoring.duplicated().sum()

54

In [101]:
# summary of the numeric fields
cc_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [102]:
# proportion of negative days_employed
cc_scoring[cc_scoring['days_employed'] < 0].shape[0]/cc_scoring.shape[0]

0.7389547038327526

In [103]:
# proportion of negative children
cc_scoring[cc_scoring['children'] < 0].shape[0]/cc_scoring.shape[0]

0.002183507549361208

In [104]:
cc_scoring['total_income'].shape, cc_scoring[~cc_scoring.index.isin(cc_scoring[cc_scoring.duplicated()].index)].shape

((21525,), (21471, 12))

In [105]:
# as days_employed and total_income in the duplicates are 0, it has no impact
# on the mean even after removal
cc_scoring[~cc_scoring.index.isin(cc_scoring[cc_scoring.duplicated()].index)].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21471.000000,19351.000000,21471.000000,21471.000000,21471.000000,21471.000000,19351.000000
mean,0.539565,63046.497661,43.279074,0.817195,0.973685,0.081086,26787.568355
std,1.382978,140827.311974,12.574291,0.548508,1.421082,0.272974,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [106]:
# delete duplicates
#cc_scoring_dup_free = cc_scoring.drop_duplicates()

In [107]:
#cc_scoring_dup_free.shape

### Conclusion

1) There are 21525 records and 12 attributes <br/>
2) `days_employed` and `total_income` have 2174 records missing respectively.<br/>
3) Fields such as `children`, `dob_years`, `education_id`, `family_status_id`, and `debt` can be converted to unsigned int to save space<br/>
4)`days_employed` and `total_income` can be rounded off to the nearest integers.<br/>
5) Minimum `dob_years` is 0, which needs to be investigated. However, this field can be categorized.<br/>
6) `days_employed` has negative values. Is this by design?<br/>
7) `education` field has inconsitent labeling.<br/>
8) `purpose` appears to be free text, which could be treated to make it useful<br/>
9) In databases, to avoid redundancy, `education` would be stored separately, and it will be linked to the credit report table using the `education_id`; the same goes for `family_status`.<br/>
10) There are 54 duplicate records.<br/>
11) The `children` field shows families with -1 children all the way up to 20 - a wide range.<br/>
12) The `days_employed` field swings wildly. 74% of the records show this discrepancy; the maximum number of years a person has ever worked is almost 1100 years. Probably Count Dracula is in this dataset.

## Data preprocessing

### Processing missing values and/or unsual values

In [108]:
# what does the distrbution of missing total_income records look like?
cc_missing = cc_scoring[cc_scoring['days_employed'].isnull()]

In [109]:
cc_missing.shape

(2174, 12)

In [110]:
rest_of_pop = cc_scoring.loc[~cc_scoring.index.isin(cc_missing.index)]

In [111]:
rest_of_pop.shape

(19351, 12)

In [112]:
# general population
rest_of_pop['dob_years'].mean(), rest_of_pop['dob_years'].median()

(43.25533564156891, 42.0)

In [113]:
# missing total income
cc_missing['dob_years'].mean(), cc_missing['dob_years'].median()

(43.632014719411224, 43.0)

In [114]:
# distribution of education in rest of population
rest_of_pop['education'].value_counts()

secondary education    12342
bachelor's degree       4222
SECONDARY EDUCATION      705
Secondary Education      646
some college             613
BACHELOR'S DEGREE        251
Bachelor's Degree        243
primary education        231
Some College              40
SOME COLLEGE              22
PRIMARY EDUCATION         16
Primary Education         14
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64

In [115]:
# distribution of education in the missing pool
cc_missing['education'].value_counts()

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
SOME COLLEGE              7
Some College              7
PRIMARY EDUCATION         1
Primary Education         1
Name: education, dtype: int64

In [116]:
# distribution of family_status in trest of the population
rest_of_pop['family_status'].value_counts()

married              11143
civil partnership     3735
unmarried             2525
divorced              1083
widow / widower        865
Name: family_status, dtype: int64

In [117]:
# distribution of family_status in the missing pool
cc_missing['family_status'].value_counts()

married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64

In [118]:
# distribution of income_type in the rest of the population
rest_of_pop['income_type'].value_counts()

employee                       10014
business                        4577
retiree                         3443
civil servant                   1312
unemployed                         2
student                            1
entrepreneur                       1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [119]:
# distribution of income_type in the missing pool
cc_missing['income_type'].value_counts()

employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

In [120]:
# proportion of debt in the rest of the population
rest_of_pop['debt'].value_counts(normalize=True)

0    0.918816
1    0.081184
Name: debt, dtype: float64

In [121]:
# proportion of debt in the missing pool
cc_missing['debt'].value_counts(normalize=True)

0    0.921803
1    0.078197
Name: debt, dtype: float64

In [122]:
# fix the negative values in `days_employed`
cc_scoring['days_employed'] = cc_scoring['days_employed'].apply(lambda x: x * -1 if x < 0 else x)

In [123]:
cc_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,66914.728907,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,139030.880527,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [124]:
# fix negative value in children
cc_scoring['children'] = cc_scoring['children'].apply(lambda x: x * -1 if x < 0 else x)

In [125]:
cc_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.543275,66914.728907,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.379876,139030.880527,12.574584,0.548138,1.420324,0.272661,16475.450632
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [126]:
# Reasonable to assume that loans are disbursed to indivuals over a certain age.
# Let us assume that the minimum age is 18 years
cc_scoring[cc_scoring['dob_years'] < 18].shape[0]

101

In [127]:
# count of dob_years < 18 years; there is only 1 value, which is 0
cc_scoring[cc_scoring['dob_years'] < 18]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

In [128]:
# mean income in this group is 25,334
cc_scoring[cc_scoring['dob_years'] < 18]['total_income'].mean(skipna=True)

25334.07289010989

In [129]:
# A reasonoable retirement age the world over is 60 years, which, in days, is 23741.
# Further, on average a person might work for 40 years +/- 3 years assuming they start at 
# age 20 years. 1 year = 365.25 days. I added another 4 years since most of the borrowers
# have a high school certificate
# Let us see how many fall outside this threshold
gt_44_yrs = cc_scoring[cc_scoring['days_employed'] > 16071]

In [130]:
gt_44_yrs.shape

(3450, 12)

In [131]:
# 16% have worked more than 40 years after completion
# of their education at, say, 20 years
gt_44_yrs.shape[0]/cc_scoring.shape[0]

0.1602787456445993

In [132]:
gt_44_yrs['dob_years'].describe()

count    3450.000000
mean       59.127536
std         7.575691
min         0.000000
25%        56.000000
50%        60.000000
75%        64.000000
max        74.000000
Name: dob_years, dtype: float64

In [133]:
# age of these individuals
gt_44_yrs[gt_44_yrs['dob_years'] >= 18]['dob_years'].describe()

count    3433.000000
mean       59.420332
std         6.346044
min        22.000000
25%        56.000000
50%        60.000000
75%        64.000000
max        74.000000
Name: dob_years, dtype: float64

In [134]:
# 17 borrowers, most of whom are retirees with only 1 debtor and
gt_44_yrs[gt_44_yrs['dob_years'] < 18]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Secondary Education,1,married,0,F,retiree,0,11406.644,car
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1175,0,370879.508002,0,secondary education,1,married,0,F,retiree,0,50231.975,to get a supplementary education
1898,0,370144.537021,0,secondary education,1,widow / widower,2,F,retiree,0,20384.043,to buy a car
4922,0,336516.005867,0,bachelor's degree,0,widow / widower,2,F,retiree,1,29369.017,to own a car
7034,0,366067.781030,0,bachelor's degree,0,unmarried,4,F,retiree,0,42099.372,education
8061,0,366457.872613,0,bachelor's degree,0,unmarried,4,F,retiree,0,9888.684,university education
10188,0,371665.278622,0,secondary education,1,married,0,M,retiree,0,16419.472,real estate transactions
12062,0,332185.354511,0,secondary education,1,married,0,F,retiree,0,33075.037,purchase of the house
12729,0,355109.725856,0,secondary education,1,widow / widower,2,F,retiree,0,8770.519,education


In [135]:
# how do they compare with those who have worked
# a reasonable number of years

others = cc_scoring[(cc_scoring['dob_years'] >= 18) &
                    ~(cc_scoring.index.isin(gt_44_yrs.index))]
others.shape

(17991, 12)

In [136]:
others['days_employed'].describe()

count    15827.000000
mean      2349.103509
std       2290.400924
min         24.141633
25%        755.511039
50%       1630.312043
75%       3156.149216
max      15835.725775
Name: days_employed, dtype: float64

In [137]:
# the group in the gt_44_years is on average older by 19 years
# compared with the group in the others df
others['dob_years'].describe()

count    17991.000000
mean        40.459118
std         10.644818
min         19.000000
25%         32.000000
50%         40.000000
75%         48.000000
max         75.000000
Name: dob_years, dtype: float64

In [138]:
# lets create age groups as income will vary with age and experience
def age_groups(age):
    if age < 18:
        return '<18 years'
    if age >=18 and age < 30:
        return '18-29 years'
    if age >=30 and age < 40:
        return '30-39 years'
    if age >=40 and age < 50:
        return '40-49 years'
    if age >=50 and age < 60:
        return '50-59 years'
    return '60+ years'

In [139]:
# create a new field
cc_scoring['age_groups'] = cc_scoring['dob_years'].apply(age_groups)

In [140]:
# distribution of ages
cc_scoring['age_groups'].value_counts(dropna=False)

30-39 years    5674
40-49 years    5371
50-59 years    4679
18-29 years    3183
60+ years      2517
<18 years       101
Name: age_groups, dtype: int64

In [141]:
#roughly in each category, there is 10x drop
cc_scoring[cc_scoring['total_income'].isnull()]['age_groups'].value_counts()

30-39 years    565
40-49 years    537
50-59 years    501
18-29 years    299
60+ years      262
<18 years       10
Name: age_groups, dtype: int64

In [142]:
# what does the mean income look like across these age categories
cc_scoring.groupby('age_groups')['total_income'].mean()

age_groups
18-29 years    25533.960641
30-39 years    28312.479963
40-49 years    28551.375635
50-59 years    25811.700327
60+ years      23021.639994
<18 years      25334.072890
Name: total_income, dtype: float64

In [143]:
# before the change
cc_scoring['total_income'].describe()

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

In [144]:
# create a dictionary mapping age_group to mean income
age_income = cc_scoring.groupby('age_groups')['total_income'].mean().to_dict()

In [145]:
def replace_missing_values(age, f):
    if age == '<18 years':
        return f[age]
    if age == '18-29 years':
        return f[age]
    if age == '30-39 years':
        return f[age]
    if age == '40-49 years':
        return f[age]
    if age == '50-59 years':
        return f[age]
    return f[age]

In [146]:
# replace missing income with mean in each age category
cc_scoring.loc[cc_scoring['total_income'].isnull(), 'total_income'] = (
    cc_scoring.loc[cc_scoring['total_income'].isnull(), 'age_groups'].apply(replace_missing_values, args=(age_income,))
)

In [147]:
# after the change
cc_scoring['total_income'].describe()

count     21525.000000
mean      26784.957102
std       15632.667343
min        3306.762000
25%       17247.708000
50%       24596.980000
75%       31286.979000
max      362496.645000
Name: total_income, dtype: float64

In [148]:
# what does the mean days_days_employed look like across these age categories
cc_scoring.groupby('age_groups')['days_employed'].mean()

age_groups
18-29 years      2082.493898
30-39 years      4155.029251
40-49 years     12383.580460
50-59 years    132907.545543
60+ years      286544.143436
<18 years       69516.105102
Name: days_employed, dtype: float64

In [149]:
#roughly in each category, there is 10x drop
cc_scoring[cc_scoring['days_employed'].isnull()]['age_groups'].value_counts()

30-39 years    565
40-49 years    537
50-59 years    501
18-29 years    299
60+ years      262
<18 years       10
Name: age_groups, dtype: int64

In [150]:
# before the change
cc_scoring['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [151]:
# create dictionary mapping age_groups to mean number of days employed
age_days_employed = cc_scoring.groupby('age_groups')['days_employed'].mean().to_dict()

In [152]:
# replace missing days_employed with mean in each age category
cc_scoring.loc[cc_scoring['days_employed'].isnull(), 'days_employed'] = (
    cc_scoring.loc[cc_scoring['days_employed'].isnull(), 'age_groups'].apply(replace_missing_values, args=(age_days_employed,))
)

In [153]:
# after the change
cc_scoring['days_employed'].describe()

count     21525.000000
mean      67216.890403
std      135283.571921
min          24.141633
25%        1025.608174
50%        2490.771162
75%        7811.054008
max      401755.400475
Name: days_employed, dtype: float64

In [154]:
# create a mask to filter those less than 18 and who have worked more than 16071 days or 44 years
# replace these abnormal records with dob_years as 60 and maximum days employed to 44 years
# This assumes the person started working at age 17 years
mask = cc_scoring[(cc_scoring['dob_years'] < 18) & (cc_scoring['days_employed'] > 16071)]
cc_scoring.loc[mask.index, 'dob_years'] = 60
cc_scoring.loc[mask.index, 'days_employed'] = 16071

In [155]:
# 2002 rows
# 15340 days = 42 years of employment assuming the individual
# started at 17-18 years
mask = cc_scoring[(cc_scoring['age_groups'] == '50-59 years') &
           (cc_scoring['days_employed'] > 15340)]
cc_scoring.loc[mask.index, 'days_employed'] = 15340

In [156]:
# 2030 rows
# 16071 days = 44 years of employment assuming the individual
# started at 17-18 years
mask = cc_scoring[(cc_scoring['age_groups'] == '60+ years') &
           (cc_scoring['days_employed'] > 16071)]
cc_scoring.loc[mask.index, 'days_employed'] = 16071

In [157]:
cc_scoring['days_employed'].describe()

count     21525.000000
mean       7952.417204
std       32073.643971
min          24.141633
25%        1025.608174
50%        2490.771162
75%        7811.054008
max      400954.247845
Name: days_employed, dtype: float64

In [158]:
# there is another sneaky group between 18-50 years
# who have abnormal number of days_employed. 165 borrowers
# to be precise
cc_scoring[cc_scoring['days_employed'] > 16071].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000
mean,0.496970,366427.333562,43.218182,0.854545,0.866667,0.096970,25747.660091
std,1.691529,22418.013246,5.876859,0.520958,1.299781,0.296817,17316.392010
min,0.000000,328734.923996,22.000000,0.000000,0.000000,0.000000,3392.845000
25%,0.000000,347311.973563,40.000000,1.000000,0.000000,0.000000,15086.342000
50%,0.000000,366777.226752,45.000000,1.000000,0.000000,0.000000,21807.668000
75%,1.000000,389397.167577,48.000000,1.000000,1.000000,0.000000,31783.901000
max,20.000000,400954.247845,49.000000,3.000000,4.000000,1.000000,109008.094000


In [159]:
cc_scoring[cc_scoring['days_employed'] > 16071]['age_groups'].value_counts()

40-49 years    128
30-39 years     30
18-29 years      7
Name: age_groups, dtype: int64

In [160]:
# what does the mean days_employed look like for these groups
cc_scoring.groupby('age_groups')['days_employed'].mean()

age_groups
18-29 years     2082.493898
30-39 years     4155.029251
40-49 years    12383.580460
50-59 years     8423.682378
60+ years      13686.214127
<18 years       5908.364429
Name: days_employed, dtype: float64

In [161]:
# recompute mean of group who have worked more than 44 years, and who fall
# into the 18-29 years, 30-39 years and 40-49 years bucket
new_mean_age_days_emp = cc_scoring[~(cc_scoring['days_employed']  > 16071)].groupby('age_groups')['days_employed'].mean()

In [162]:
# replace these incorrect values with the mean days worked for each age category
cc_scoring.loc[cc_scoring['days_employed'] > 16071, 'days_employed'] = (
    cc_scoring.loc[cc_scoring['days_employed']  > 16071, 'age_groups'].apply(replace_missing_values, args=(new_mean_age_days_emp,))
)

In [163]:
# recompute the age_groups
cc_scoring['age_groups'] = cc_scoring['dob_years'].apply(age_groups)

In [164]:
# after cleaning
cc_scoring.groupby('age_groups')['days_employed'].mean()

age_groups
18-29 years     1292.074716
30-39 years     2239.499525
40-49 years     3721.638383
50-59 years     8423.682378
60+ years      13711.524354
<18 years       2200.375775
Name: days_employed, dtype: float64

In [165]:
# 74 rows where the age is <18 years and who aren't retirees
# will have to ignore these observations
cc_scoring[cc_scoring['age_groups'] == '<18 years']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_groups
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions,<18 years
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation,<18 years
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car,<18 years
1149,0,934.654854,0,secondary education,1,married,0,F,business,0,32296.389,buy real estate,<18 years
1386,0,5043.219890,0,bachelor's degree,0,married,0,M,civil servant,0,38483.779,purchase of a car,<18 years
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18732,2,986.789230,0,bachelor's degree,0,civil partnership,1,F,business,0,45972.566,education,<18 years
18851,0,10637.001569,0,secondary education,1,married,0,F,civil servant,0,21845.941,supplementary education,<18 years
19371,1,2356.250615,0,bachelor's degree,0,married,0,F,business,0,12469.795,car,<18 years
21179,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate,<18 years


### Conclusion

1) By comparing the borrowers with missing income with those with non-missing income, we see that there is only a small difference in their mean/median `ages`, proportion of `debt`; In the `education` and `family_status` categories, the proportion is very similar. Further, the deviation between median income and mean is a little of a $1000. Looking at the age_group wise mean income, there isn't a significant variation. However, in the 60+ years category, we can see a relative drop by a few thousad dollars. Replacing the missing `total_income` by age_groups wise mean hasn't altered the overall mean significantly. So, we are good.<br/>

2) Roughly 20% (increases from 16% after filling in missing values with their respective means) have worked more than 44 years, which is a lot of man years in a lifetime. There are unrealistic numbers at both ends of the spectrum ranging from 0 years to 1100 years. To fill in the missing value for `days_worked`, we will fill them with age_group wise mean`.<br/>

3) The `days_employed` field has outliers in all age_groups: <18 year, 20-29 years, 30-39 years, 40-49 years, 50-59 years, and 60+ years. Thos supposedly less than 18 years have their actual reported age as 0. But we can also see that these borrowers are retired. Therefore, we can replace the `days_employed` by making an assumption that the individual has worked 44 years since the age of 17-18. And their ages can be replaced with the assumed retirement age of 60 years.<br/>

4) For those who are in the 50 - 75 year, we can replace the outliers with 2 sets of values; those in the 50-59 years will be assumed to have worked for 42 years, and those in the 60+ years group for 44 years. Ofcourse, this might not be the best solution, but the other alternative is to drop them.<br/>

5) For those in the 18-29 years, 30-39 years, and 40-49 years, a new mean number of `days_employed` was computed, and this was used to replace the outliers.

6) Despite fixing issues, there are 74 observations where the age is 0 years; these will have to be ignore or dropped. Or these can be inferred from the `days_employed` field.

7) The missing values in these 2 fields seem to be missing at random; they are representative of the `family_status`, `education`, `income_type`, `total_income`, and `purpose`. This is possible if the application didn't have any validation checks in place, which allowed the user to enter incorrect values. Or it could have been a system error.


### Data type replacement

In [166]:
cc_scoring['days_employed'] = np.round(cc_scoring['days_employed']).astype(int)
cc_scoring['total_income'] = np.round(cc_scoring['total_income']).astype(int)
cc_scoring['education_id'] = cc_scoring['education_id'].astype(np.uint8)
cc_scoring['family_status_id'] = cc_scoring['education_id'].astype(np.uint8)
cc_scoring['debt'] = cc_scoring['debt'].astype(np.uint8)
cc_scoring['children'] = cc_scoring['children'].astype(np.uint8)
cc_scoring['dob_years'] = cc_scoring['dob_years'].astype(np.uint8)

In [167]:
cc_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null uint8
days_employed       21525 non-null int64
dob_years           21525 non-null uint8
education           21525 non-null object
education_id        21525 non-null uint8
family_status       21525 non-null object
family_status_id    21525 non-null uint8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null uint8
total_income        21525 non-null int64
purpose             21525 non-null object
age_groups          21525 non-null object
dtypes: int64(2), object(6), uint8(5)
memory usage: 1.4+ MB


### Conclusion

1) We have successfully reduced the memory usage by ~600KB or about 30%

2) Converting from float to int makes sense here as `days_employed` and `total_income` can be rounded off to the nearest integer. We don't require precision to report such fields.

3) For the integer fields which were converted, again, these are just ids; therefore to save space, we changed the data type of unsinged int.

### Processing duplicates

In [168]:
cc_scoring['education'] = cc_scoring['education'].str.lower()

In [169]:
cc_scoring.duplicated().sum()

71

In [170]:
cc_scoring.drop_duplicates(inplace=True)

### Conclusion

1) There are 71 duplicates after changing the case for `education` - an increase of 17 duplicates, which were removed.

2) The duplicates probably occurred as there is no unique id to identify the borrower. Generally, having a customer id, and adding a constraint to this will help ensure duplicates are not entered.

### Categorizing Data

In [171]:
# distribution of education
cc_scoring['education'].value_counts()

secondary education    15172
bachelor's degree       5250
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

In [172]:
# unique education
cc_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [173]:
# Graduate Degree, Primary Education, Some College, Bachelor's Degree, Seconday Education
cc_scoring['education'] = cc_scoring['education'].str.capitalize()

In [174]:
# proportion of education after cleaning
cc_scoring['education'].value_counts(normalize=True)

Secondary education    0.707187
Bachelor's degree      0.244710
Some college           0.034679
Primary education      0.013144
Graduate degree        0.000280
Name: education, dtype: float64

In [175]:
# proportion of family_status
cc_scoring['family_status'].value_counts(normalize=True)

married              0.575138
civil partnership    0.193484
unmarried            0.130978
divorced             0.055701
widow / widower      0.044700
Name: family_status, dtype: float64

In [176]:
# proportion of gender
cc_scoring['gender'].value_counts(normalize=True)

F      0.660669
M      0.339284
XNA    0.000047
Name: gender, dtype: float64

In [177]:
# proportion of income_type
cc_scoring['income_type'].value_counts(normalize=True)

employee                       0.516640
business                       0.236692
retiree                        0.178475
civil servant                  0.067913
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000047
paternity / maternity leave    0.000047
Name: income_type, dtype: float64

In [178]:
# distribution of purpose
cc_scoring['purpose'].value_counts()

wedding ceremony                            791
having a wedding                            768
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
buying property for renting out             651
transactions with commercial real estate    650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                635
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             621
purchase of my own house                    620
building a property                         619
housing renovation                          607
buy residential real estate                 606
buying my own car                       

In [179]:
# unique purposes
cc_scoring['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [180]:
def clean_purpose_field(purpose):
    """
    Clean up the many variations of loan purpose
    to a manageable number
    """
    keywords = {
                'car': 'Vehicle loan',
                'cars': 'Vehicle loan',
                'education': 'Education loan',
                'educated': 'Education loan',
                'university': 'Education loan',
                'house': 'Housing loan',
                'wedding': 'Personal loan',
                'estate': 'Housing loan',
                'property': 'Housing loan',
                'housing': 'Housing loan'
    }
    match = set()
    for word in purpose.lower().split():
        if word in keywords:
            match.add(keywords[word])
    return list(match)[0] if len(match) > 0 else 'Unknown'

In [181]:
#key words
# house, car, education, wedding, real estate, property, house renovation
cc_scoring['loan_category'] = cc_scoring['purpose'].apply(clean_purpose_field)

In [182]:
# check if count matches the total records in the df
cc_scoring['loan_category'].value_counts().sum()

21454

In [183]:
# proportion of loan category after cleaning
cc_scoring['loan_category'].value_counts(normalize=True)

Housing loan      0.503915
Vehicle loan      0.200708
Education loan    0.187051
Personal loan     0.108325
Name: loan_category, dtype: float64

### Conclusion

1) The `loan purpose` field looks much cleaner after cleaning; we have only 4 categories which represents the 
various loan purposes. This will save time for the loan manager.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [184]:
# results are identical to using crosstab. I would need to 
# perform another step to get into a table format.
cc_scoring.groupby(['children', 'debt'])['debt'].count()

children  debt
0         0       13028
          1        1063
1         0        4410
          1         445
2         0        1858
          1         194
3         0         303
          1          27
4         0          37
          1           4
5         0           9
20        0          68
          1           8
Name: debt, dtype: int64

In [185]:
# only field close to paying a loan is the debt field; there is no element of loan duration
res = pd.crosstab(index=cc_scoring['children'], 
            columns=cc_scoring['debt']).reset_index().rename(columns={0: 'loan_paid', 
                                                                              1:'loan_default'}
                                                            )
res['percent_paid'] = 100 * res['loan_paid'] / (res['loan_paid'] + res['loan_default'])

In [186]:
res

debt,children,loan_paid,loan_default,percent_paid
0,0,13028,1063,92.456178
1,1,4410,445,90.834192
2,2,1858,194,90.545809
3,3,303,27,91.818182
4,4,37,4,90.243902
5,5,9,0,100.000000
6,20,68,8,89.473684


### Conclusion

In [187]:
# median total income for the families with 20 kids is not very
# different from the median of the population
cc_scoring[cc_scoring['children'] == 20]['total_income'].median()

24110.5

In [188]:
# median income of the population
cc_scoring['total_income'].median()

24567.0

In [193]:
# borrowers with 20 families and their loan purpose preference
cc_scoring[cc_scoring['children'] == 20]['loan_category'].value_counts(normalize=True)

Housing loan      0.473684
Vehicle loan      0.210526
Education loan    0.197368
Personal loan     0.118421
Name: loan_category, dtype: float64

In [194]:
# default in this group is 2% higher than the rest of the population
cc_scoring[cc_scoring['children'] == 20]['debt'].value_counts(normalize=True)

0    0.894737
1    0.105263
Name: debt, dtype: float64

- Is there a relation between marital status and repaying a loan on time?

In [89]:
res = pd.crosstab(index=cc_scoring['family_status'], 
            columns=cc_scoring['debt']).reset_index().rename(columns={0: 'loan_paid', 
                                                                              1:'loan_default'}
                                                            )
res['percent_paid'] = 100 * res['loan_paid'] / (res['loan_paid'] + res['loan_default'])

In [90]:
res

debt,family_status,loan_paid,loan_default,percent_paid
0,civil partnership,3763,388,90.652855
1,divorced,1110,85,92.887029
2,married,11408,931,92.454818
3,unmarried,2536,274,90.249110
4,widow / widower,896,63,93.430657


### Conclusion

1) All I can conclude from this is that the widow category are relatively better than the other categoris in paying
off their debt on time. Relatively, the civil partnership and unmarried categories have a slightly poorer chance of paying on time. We need more data to actually determine which group has a higher chance of defaulting.

- Is there a relation between income level and repaying a loan on time?

In [95]:
cc_scoring.groupby('debt')['total_income'].mean()

debt
0    26839.808920
1    26175.049971
Name: total_income, dtype: float64

### Conclusion

1) Judging by the mean income in both categories, it seems to suggest that there is no clear distinction in the `total_income` of debtors versus non-debtors. In reality, most NPAs are triggered by wealthy individuals through scams.

- How do different loan purposes affect on-time repayment of the loan?

In [96]:
res = pd.crosstab(index=cc_scoring['loan_category'], 
            columns=cc_scoring['debt']).reset_index().rename(columns={0: 'loan_paid', 
                                                                              1:'loan_default'}
                                                            )
res['percent_paid'] = 100 * res['loan_paid'] / (res['loan_paid'] + res['loan_default'])

In [97]:
res

debt,loan_category,loan_paid,loan_default,percent_paid
0,Education loan,3644,370,90.782262
1,Housing loan,10032,782,92.768633
2,Personal loan,2149,186,92.034261
3,Vehicle loan,3905,403,90.645311


### Conclusion

1) Here too, we can see that the borrowers in the Personal loan and Housing loan categories are more prompt at paying their loans when compared with those in the Education loan and Vehicle loan sections.

## General conclusion

1) The mean age in the group is 43 years; Majority of the borrowers are women - 66%.

2) The most common `education` is secondary education at 70%.

3) The most popular `income_type` is employee who make up 51%  of the population.

4) 58% of the population is married.

5) After cleaning the mean `days_employed` ranges from 3 years to 44 years.

6) After cleaning the mean `total_income is 26700 per month.
1) Based on the above observations, it appears that no one field significantly impacts loan repayment. Maybe they
have to be looked at collectively. Additionally, this dataset maybe lacking important features such as credit score, 
previous loan history, location, loan size, duration, etc which may have a greater impact on loan repayment.

<div style="border:solid Chocolate 2px; padding: 40px">

**The overall review conclusion: second iteration of the review**

Sandeep, thank you for submitting your project with corrections! And thank you for your feedback and your position, it is great to collaborate with an analyst with the strong position! 
    
I hasten to say that your project has been accepted and you can proceed to the next sprint!
    
**I wish you exciting and cool projects in the next sprints😉**

![gif](https://media.giphy.com/media/12XDYvMJNcmLgQ/giphy.gif)  

## Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [ ]  file examined;
- [ ]  missing values defined;
- [ ]  missing values are filled;
- [ ]  an explanation of which missing value types were detected;
- [ ]  explanation for the possible causes of missing values;
- [ ]  an explanation of how the blanks are filled;
- [ ]  replaced the real data type with an integer;
- [ ]  an explanation of which method is used to change the data type and why;
- [ ]  duplicates deleted;
- [ ]  an explanation of which method is used to find and remove duplicates;
- [ ]  description of the possible reasons for the appearance of duplicates in the data;
- [ ]  data is categorized;
- [ ]  an explanation of the principle of data categorization;
- [ ]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [ ]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [ ]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [ ]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [ ]  conclusions are present on each stage;
- [ ]  a general conclusion is made.